In [1]:
# Install necessary packages
!pip install ultralytics wandb
!pip install -U ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 583.6 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.8/792.8 kB 5.2 MB/s eta 0:00:0000:0100:01
^C
ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 1.2 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 19.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab-widgets 3.0.9
    Uninstalling jupyterlab-widgets-3.0.9:
      Successfully uninstalled jupyterlab-widgets-3.0.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.

In [2]:
import yaml

# Define the path to the existing data.yaml file
yaml_path = '/kaggle/input/yolov8-dataset/Dataset/data.yaml'

# Read the content of the data.yaml file
with open(yaml_path, 'r') as file:
    data_config = yaml.safe_load(file)

# Modify the paths in the data.yaml content
data_config['path'] = '/kaggle/input/yolov8-dataset/Dataset'
data_config['train'] = '/kaggle/input/yolov8-dataset/Dataset/train/images'
data_config['val'] = '/kaggle/input/yolov8-dataset/Dataset/val/images'

# Save the modified content to the working directory
modified_yaml_path = '/kaggle/working/data.yaml'
with open(modified_yaml_path, 'w') as file:
    yaml.safe_dump(data_config, file)

print(f"Modified data.yaml file saved at: {modified_yaml_path}")


Modified data.yaml file saved at: /kaggle/working/data.yaml


In [3]:
# Import libraries
import wandb
from ultralytics import YOLO

# Login to wandb
wandb.login()

# Define your dataset and model configuration
data_config = '/kaggle/working/data.yaml'  # Update with your dataset path
model_config = 'yolov8n.yaml'

# Initialize the model
model = YOLO(model_config)

# Train the model with early stopping
model.train(
    data=data_config,
    epochs=100,  # Adjust the number of epochs as needed
    project='Traffic_Analysis',  # Change this to your project name
    name='First',  # Change this to your run name
    exist_ok=True,
    patience=10  # Number of epochs with no improvement after which training will be stopped
)

# Save the model after training
model.save('/kaggle/working/trained_model.pt')

ModuleNotFoundError: No module named 'ultralytics'